# Pricing and Risk Calculation - Portfolios

Portfolios allow for efficient pricing and risk. The same principles in the in the basic risk and pricing tutorials can be applied to portfolios. Pricing and risks can be viewed for an individual instrument or at the aggregate portfolio level

In [ ]:
from gs_quant.session import Environment, GsSession
from gs_quant.common import PayReceive
from gs_quant.instrument import IRSwaption
import gs_quant.risk as risk
import datetime as dt

In [ ]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

In [ ]:
swaption1 = IRSwaption(PayReceive.Receive, '5y', 'EUR',
                       expiration_date='3m', name='EUR3m5y')
swaption2 = IRSwaption(PayReceive.Receive, '5y', 'EUR',
                       expiration_date='6m', name='EUR6m5y')

In [ ]:
# Create Portfolio w/ swaptions
from gs_quant.markets.portfolio import Portfolio
portfolio = Portfolio((swaption1, swaption2))
portfolio.resolve()

In [ ]:
# Calculate Risk for Portfolio
port_risk = portfolio.calc(
    (risk.DollarPrice, risk.IRDeltaParallel, risk.IRVega))

In [ ]:
# View Instrument Level risk for portfolio
port_risk[risk.IRDeltaParallel]['EUR3m5y']

In [ ]:
# View Aggregated Risk for a Portfolio
port_risk[risk.IRDeltaParallel].aggregate()

In [ ]:
# Calculate Portfolio Risks within HistoricalPricingContext
from gs_quant.markets import HistoricalPricingContext

start_date = dt.date(2019, 6, 3)
end_date = dt.date(2019, 11, 18)

with HistoricalPricingContext(start_date, end_date):
    port_ird_res = portfolio.calc(risk.IRDeltaParallelLocalCcy)

In [ ]:
port_ird_agg = port_ird_res[risk.IRDeltaParallelLocalCcy].aggregate()
swap1_ird = port_ird_res[risk.IRDeltaParallelLocalCcy]['EUR3m5y']
swap2_ird = port_ird_res[risk.IRDeltaParallelLocalCcy]['EUR6m5y']

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.xlabel('Date')

ax1 = port_ird_agg.plot(label='Portfolio')
swap1_ird.plot(label='EUR3m5y')
swap2_ird.plot(label='EUR6m5y')
h1, l1 = ax1.get_legend_handles_labels()
plt.title('IRDeltaParallel Timeseries')
plt.legend(h1, l1, loc=2)
plt.show()